In [2]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
from tqdm import tqdm

In [3]:
#file_path = '../../data/BPI Challenge 2017.xes'
#file_path = '../../data/InternationalDeclarations.xes'
#file_path = '../../data/BPI_Challenge_2019.xes'
#file_path = '../../data/BPI Challenge 2018.xes'
file_path = '../../data/BPI_Challenge_2012.xes'
event_log = pm4py.read_xes(file_path)

parsing log, completed traces ::   0%|          | 0/13087 [00:00<?, ?it/s]

In [4]:
event_log

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,112,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
1,112,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
2,112,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
3,112,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
...,...,...,...,...,...,...,...
262195,112,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+00:00,2012-02-29 23:51:16.799000+00:00,214376,15000
262196,112,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+00:00,2012-02-29 23:51:16.799000+00:00,214376,15000
262197,11169,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+00:00,2012-02-29 23:51:16.799000+00:00,214376,15000
262198,11169,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+00:00,2012-02-29 23:51:16.799000+00:00,214376,15000


In [5]:
event_log['lifecycle:transition'].unique()

array(['COMPLETE', 'SCHEDULE', 'START'], dtype=object)

In [4]:
event_log[(event_log['case:concept:name'] == '173688') & (event_log['concept:name'] == 'W_Nabellen offertes')].sort_values('time:timestamp')

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
10,NaN,SCHEDULE,W_Nabellen offertes,2011-10-01 11:45:11.554000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
12,NaN,START,W_Nabellen offertes,2011-10-01 12:15:41.290000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
13,NaN,COMPLETE,W_Nabellen offertes,2011-10-01 12:17:08.924000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
14,10913,START,W_Nabellen offertes,2011-10-08 16:26:57.720000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
15,10913,COMPLETE,W_Nabellen offertes,2011-10-08 16:32:00.886000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
16,11049,START,W_Nabellen offertes,2011-10-10 11:32:22.495000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
19,11049,COMPLETE,W_Nabellen offertes,2011-10-10 11:33:05.791000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000


In [5]:
pandas.unique(event_log['case:concept:name']).shape

(13087,)

In [6]:
merged_event_log = pandas.merge(event_log, event_log,
                                left_on=['case:concept:name', 'org:resource', 'concept:name'],
                                right_on=['case:concept:name', 'org:resource', 'concept:name'],
                                suffixes=('_start', '_complete'))
start_end_event_log = merged_event_log[(merged_event_log['lifecycle:transition_start'] == 'START') & (merged_event_log['lifecycle:transition_complete'] == 'COMPLETE')]
start_end_event_log.loc[:, 'duration'] = start_end_event_log['time:timestamp_complete'] - start_end_event_log['time:timestamp_start']
start_end_event_log.loc[:, 'duration_seconds'] =  (start_end_event_log['duration']).astype('timedelta64[s]').astype(int)
start_end_event_log = start_end_event_log[start_end_event_log['duration_seconds'] > 0]
start_end_event_log

/tmp/ipykernel_76109/3138905879.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.loc[:, 'duration'] = start_end_event_log['time:timestamp_complete'] - start_end_event_log['time:timestamp_start']
/tmp/ipykernel_76109/3138905879.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.loc[:, 'duration_seconds'] =  (start_end_event_log['duration']).astype('timedelta64[s]').astype(int)


,org:resource,lifecycle:transition_start,concept:name,time:timestamp_start,case:REG_DATE_start,case:concept:name,case:AMOUNT_REQ_start,lifecycle:transition_complete,time:timestamp_complete,case:REG_DATE_complete,case:AMOUNT_REQ_complete,duration,duration_seconds
5,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,COMPLETE,2011-10-01 11:45:13.917000+00:00,2011-10-01 00:38:44.546000+00:00,20000,0 days 00:08:27.480000,507
18,NaN,START,W_Nabellen offertes,2011-10-01 12:15:41.290000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,COMPLETE,2011-10-01 12:17:08.924000+00:00,2011-10-01 00:38:44.546000+00:00,20000,0 days 00:01:27.634000,87
23,10913,START,W_Nabellen offertes,2011-10-08 16:26:57.720000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,COMPLETE,2011-10-08 16:32:00.886000+00:00,2011-10-01 00:38:44.546000+00:00,20000,0 days 00:05:03.166000,303
27,11049,START,W_Nabellen offertes,2011-10-10 11:32:22.495000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,COMPLETE,2011-10-10 11:33:05.791000+00:00,2011-10-01 00:38:44.546000+00:00,20000,0 days 00:00:43.296000,43
33,10629,START,W_Valideren aanvraag,2011-10-13 10:05:26.925000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,COMPLETE,2011-10-13 10:37:37.026000+00:00,2011-10-01 00:38:44.546000+00:00,20000,0 days 00:32:10.101000,1930
...,...,...,...,...,...,...,...,...,...,...,...,...,...
729165,11169,START,W_Afhandelen leads,2012-03-01 09:25:39.409000+00:00,2012-02-29 23:28:55.349000+00:00,214370,20000,COMPLETE,2012-03-01 09:28:21.228000+00:00,2012-02-29 23:28:55.349000+00:00,20000,0 days 00:02:41.819000,161
729173,11169,START,W_Afhandelen leads,2012-03-01 09:25:47.680000+00:00,2012-02-29 23:43:09.766000+00:00,214373,8500,COMPLETE,2012-03-01 09:27:52.388000+00:00,2012-02-29 23:43:09.766000+00:00,8500,0 days 00:02:04.708000,124
729179,10933,START,W_Completeren aanvraag,2012-03-01 20:11:14.761000+00:00,2012-02-29 23:43:09.766000+00:00,214373,8500,COMPLETE,2012-03-01 20:22:41.157000+00:00,2012-02-29 23:43:09.766000+00:00,8500,0 days 00:11:26.396000,686
729189,11119,START,W_Nabellen offertes,2012-03-10 12:46:22.700000+00:00,2012-02-29 23:43:09.766000+00:00,214373,8500,COMPLETE,2012-03-10 12:50:54.881000+00:00,2012-02-29 23:43:09.766000+00:00,8500,0 days 00:04:32.181000,272


In [7]:
start_end_event_log[start_end_event_log['case:concept:name'] == '173691'].sort_values('time:timestamp_start')

,org:resource,lifecycle:transition_start,concept:name,time:timestamp_start,case:REG_DATE_start,case:concept:name,case:AMOUNT_REQ_start,lifecycle:transition_complete,time:timestamp_complete,case:REG_DATE_complete,case:AMOUNT_REQ_complete,duration,duration_seconds
45,NaN,START,W_Completeren aanvraag,2011-10-01 11:37:32.393000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-01 11:43:13.178000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:05:40.785000,340
47,NaN,START,W_Completeren aanvraag,2011-10-01 11:37:32.393000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-01 14:35:48.791000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 02:58:16.398000,10696
55,NaN,START,W_Completeren aanvraag,2011-10-01 14:27:57.775000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-01 14:35:48.791000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:07:51.016000,471
75,NaN,START,W_Nabellen offertes,2011-10-01 14:36:13.443000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-01 14:36:25.794000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:00:12.351000,12
78,NaN,START,W_Nabellen offertes,2011-10-01 14:36:13.443000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-03 16:56:21.195000+00:00,2011-10-01 08:08:58.256000+00:00,5000,2 days 02:20:07.752000,181207
80,NaN,START,W_Nabellen offertes,2011-10-01 14:36:13.443000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-03 16:57:16.563000+00:00,2011-10-01 08:08:58.256000+00:00,5000,2 days 02:21:03.120000,181263
94,NaN,START,W_Nabellen offertes,2011-10-03 16:54:43.257000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-03 16:56:21.195000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:01:37.938000,97
96,NaN,START,W_Nabellen offertes,2011-10-03 16:54:43.257000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-03 16:57:16.563000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:02:33.306000,153
124,NaN,START,W_Nabellen offertes,2011-10-03 16:56:34.792000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-03 16:57:16.563000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:00:41.771000,41
134,11049,START,W_Nabellen offertes,2011-10-10 10:13:44.827000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-10 10:14:45.514000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:01:00.687000,60


## Exemplary case
Id: 173691

In [8]:
start_end_event_log = start_end_event_log.loc[start_end_event_log.groupby(
    ['org:resource', 'lifecycle:transition_start', 'concept:name', 'time:timestamp_start'], dropna=False)['duration'].idxmin()]

In [9]:
start_end_event_log[start_end_event_log['case:concept:name'] == '173691'].sort_values('time:timestamp_start')

,org:resource,lifecycle:transition_start,concept:name,time:timestamp_start,case:REG_DATE_start,case:concept:name,case:AMOUNT_REQ_start,lifecycle:transition_complete,time:timestamp_complete,case:REG_DATE_complete,case:AMOUNT_REQ_complete,duration,duration_seconds
45,NaN,START,W_Completeren aanvraag,2011-10-01 11:37:32.393000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-01 11:43:13.178000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:05:40.785000,340
55,NaN,START,W_Completeren aanvraag,2011-10-01 14:27:57.775000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-01 14:35:48.791000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:07:51.016000,471
75,NaN,START,W_Nabellen offertes,2011-10-01 14:36:13.443000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-01 14:36:25.794000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:00:12.351000,12
94,NaN,START,W_Nabellen offertes,2011-10-03 16:54:43.257000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-03 16:56:21.195000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:01:37.938000,97
124,NaN,START,W_Nabellen offertes,2011-10-03 16:56:34.792000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-03 16:57:16.563000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:00:41.771000,41
134,11049,START,W_Nabellen offertes,2011-10-10 10:13:44.827000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-10 10:14:45.514000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:01:00.687000,60
140,10809,START,W_Valideren aanvraag,2011-10-10 11:12:36.612000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-10 11:30:54.893000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:18:18.281000,1098
154,10809,START,W_Valideren aanvraag,2011-10-10 14:11:42.124000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-10 14:14:13.394000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:02:31.270000,151
168,10809,START,W_Valideren aanvraag,2011-10-10 14:14:52.842000+00:00,2011-10-01 08:08:58.256000+00:00,173691,5000,COMPLETE,2011-10-10 14:17:34.633000+00:00,2011-10-01 08:08:58.256000+00:00,5000,0 days 00:02:41.791000,161


In [10]:
backup = start_end_event_log.copy()
backup.to_csv('celonis_shit.csv', date_format='%Y-%m-%d %H:%M:%S')

In [11]:
tqdm.pandas()
start_end_event_log = pandas.merge(start_end_event_log, start_end_event_log,
                                left_on=['case:concept:name', 'org:resource', 'concept:name'],
                                right_on=['case:concept:name', 'org:resource', 'concept:name'],
                                suffixes=('_left', '_right')).progress_apply(lambda x: x)
start_end_event_log.loc[:, 'to_next_duration'] = start_end_event_log['time:timestamp_start_left'] - start_end_event_log['time:timestamp_start_left']
start_end_event_log.loc[:, 'to_next_duration_seconds'] =  (start_end_event_log['to_next_duration']).astype('timedelta64[s]').astype(int)
start_end_event_log = start_end_event_log[start_end_event_log['to_next_duration_seconds'] >= 0]



100%|██████████████████████████████████████████| 23/23 [00:00<00:00, 303.85it/s]


In [12]:
start_end_event_log = start_end_event_log.loc[start_end_event_log.groupby(
    ['org:resource', 'concept:name', 'time:timestamp_start_left'], dropna=False)['to_next_duration_seconds'].nsmallest(2)]#.idxmin()]

In [13]:
start_end_event_log = backup.copy()
start_end_event_log = pandas.merge(start_end_event_log, start_end_event_log,
                                left_on=['case:concept:name', 'org:resource', 'concept:name'],
                                right_on=['case:concept:name', 'org:resource', 'concept:name'],
                                suffixes=('_left', '_right')).progress_apply(lambda x: x)
start_end_event_log.loc[:, 'to_next_duration'] = start_end_event_log['time:timestamp_start_right'] - start_end_event_log['time:timestamp_start_left']
start_end_event_log.loc[:, 'to_next_duration_seconds'] =  (start_end_event_log['to_next_duration']).astype('timedelta64[s]').astype(int)
start_end_event_log = start_end_event_log[start_end_event_log['to_next_duration_seconds'] >= 0]


100%|██████████████████████████████████████████| 23/23 [00:00<00:00, 317.95it/s]


In [14]:
start_end_event_log = start_end_event_log.loc[start_end_event_log.groupby(
    ['case:concept:name', 'org:resource', 'concept:name', 'time:timestamp_start_left'], dropna=False).indices]

TypeError: Passing a dict as an indexer is not supported. Use a list instead.

In [ ]:
gb = start_end_event_log.groupby(
    ['case:concept:name', 'org:resource', 'concept:name', 'time:timestamp_start_left'], dropna=False)['to_next_duration_seconds'].nsmallest(2)

In [ ]:
gb2 = start_end_event_log.groupby(
    ['case:concept:name', 'org:resource', 'concept:name', 'time:timestamp_start_left'], dropna=False, as_index=False)

In [ ]:
start_end_event_log.set_index(['case:concept:name', 'org:resource', 'concept:name', 'time:timestamp_start_left'])[gb.index]

In [ ]:
start_end_event_log.set_index(['case:concept:name', 'org:resource', 'concept:name', 'time:timestamp_start_left'])[[204314, 10124, 'W_Afhandelen leads', '2012-01-27 07:47:23.961000+00:00']]

In [ ]:
start_end_event_log[gb2

In [ ]:
gb2.indices

In [ ]:
start_end_event_log.groupby(
    ['case:concept:name', 'org:resource', 'concept:name', 'time:timestamp_start_left'], dropna=False)]['to_next_duration_seconds'].nsmallest(1)

In [ ]:
start_end_event_log[start_end_event_log['case:concept:name'] == '173691'].sort_values('time:timestamp_start_left')

In [ ]:
start_end_event_log

## Histogram for all activities / resources

In [ ]:
start_end_event_log['duration_seconds'].hist(bins=30, range=(0, start_end_event_log['duration_seconds'].quantile(q=0.9)))

In [ ]:
start_end_event_log['duration_seconds'].quantile(q=0.99)

In [ ]:
start_end_event_log['duration_seconds'].quantile(q=0.999) / 3600

99% of the tasks have been finished below 3,6 hours
0.1% of the activities take longer than 58,5 hours -> they were started before a weekend / public holiday and finished afterwards

In [ ]:
start_end_event_log[start_end_event_log['duration_seconds'] > start_end_event_log['duration_seconds'].quantile(q=0.999)]


In [ ]:
pandas.unique(start_end_event_log['concept:name'])

In [ ]:
pandas.unique(start_end_event_log['org:resource']).shape

In [ ]:
for concept in pandas.unique(start_end_event_log['concept:name']):
    print(concept)
    concept_log = start_end_event_log[start_end_event_log['concept:name'] == concept]
    description = concept_log['duration_seconds'].describe()
    print(description)
    concept_log['duration_seconds'].hist(bins=30, range=(0,description['75%']))
    plt.show()

In [ ]:
for resource in pandas.unique(start_end_event_log['org:resource']):
    print(resource)
    resource_log = start_end_event_log[start_end_event_log['org:resource'] == resource]
    description = resource_log['duration_seconds'].describe()
    print(description)
    if not pandas.isna(resource):
        resource_log['duration_seconds'].hist(bins=100, range=(0,description['75%']))
        plt.show()

In [ ]:
for i, row in start_end_event_log[['org:resource', 'concept:name']].value_counts().reset_index(name='count').iterrows():
    resource = row['org:resource']
    concept = row['concept:name']
    concept_resource_log = start_end_event_log[(start_end_event_log['org:resource'] == resource) & (start_end_event_log['concept:name'] == concept)]
    description = concept_resource_log['duration_seconds'].describe()
    print(resource, ',', concept, ':', description['count'])

    print(description)
    concept_resource_log['duration_seconds'].hist(bins=100, range=(0,description['75%']))
    plt.show()

# Gaussian Mixture Models

- We use activity: W_Afhandelen leads
- Afterwards we split by all resources with > 15 samples + all remaining resources
- Compare: ?

In [ ]:
class GMM:
    def __init__(self, start_end_event_log, n=10, percentile=False):
        self.durations = start_end_event_log['duration_seconds'].to_numpy()
        self._get_x(start_end_event_log, percentile)
        self.gmm = self.get_gmm(self.x, n)

    def _get_x(self, durations, percentile):
        if percentile:
            self.durations = self.durations[self.durations <= np.percentile(self.durations, percentile)]
        self.x = np.ravel(self.durations).astype(np.float64).reshape(-1,1)
    
    def get_gmm(self, x, n):
        mm = GaussianMixture(n,
                             covariance_type='full',
                             n_init=10,
                             #covariance_type='spherical',
                             init_params='k-means++',
                             #init_params='kmeans',
                             #init_params='random_from_data',
                             #init_params='random',
                             max_iter=1000
                            )
        mm.fit(x)
        return mm

    def get_pdf_values(self, f_axis):
        X = np.ravel(f_axis).astype(np.float64).reshape(-1,1)
        pdf = lambda x : sum([weight * stats.norm.pdf(x, mean, np.sqrt(cov))[0] \
                              for weight, mean, cov in zip(self.gmm.weights_, self.gmm.means_, self.gmm.covariances_)])
        return np.array([pdf(x) for x in X]).flatten()

    def goodness(self):
        #predictions = self.get_pdf_values(self.x)
        #gmm_samples, _ = self.gmm.sample(100000)
        #data_hist_weights, data_hist_bins = np.histogram(self.x, bins=min(200, int(len(self.x))), density=False)
        #sample_hist_weights, sample_hist_bins  = np.histogram(gmm_samples, bins=min(200, int(len(self.x))), density=False)

        #bins_2_values = lambda bins : [(bins[i] + bins[i+1]) / 2 for i in range(len(bins)-1)]
        #data_hist_values, sample_hist_values = bins_2_values(data_hist_bins), bins_2_values(sample_hist_bins)
        #wd_hist = stats.wasserstein_distance(data_hist_values, sample_hist_values, data_hist_weights, sample_hist_weights)
        #times_enlargen = int(100000 / len(self.x))
        #augmented_x = np.array([])
        #for i in range(times_enlargen):
        #    augmented_x = np.append(augmented_x, np.ravel(self.x))
        #wd_x = stats.wasserstein_distance(augmented_x, self.gmm.sample(len(augmented_x))[0].ravel())
        #wd_pdf = stats.wasserstein_distance(np.ravel(self.x), np.ravel(self.x), predictions, np.ones(self.x.shape))
        bic = self.gmm.bic(self.x)
        return bic
        #aic = self.gmm.aic(self.x)
        #return ot.wasserstein_1d(np.ravel(self.x), np.ravel(self.x), predictions)
        return wd_x#, wd_hist, wd_pdf, bic, aic
    
    def plot_gmm(self, other_gmm=None, show_all=True, percentile=False):
        # remove outliers
        if percentile:
            pruned_durations = self.durations[self.durations <= np.percentile(self.durations, percentile)]
        else:
            pruned_durations = self.durations
        f_axis = np.linspace(pruned_durations.min()-10, pruned_durations.max(), 10000)
        predictions = self.get_pdf_values(f_axis)
        plt.plot(f_axis, predictions)
        
        gmm_samples, _ = self.gmm.sample(100000)
        plt.hist(gmm_samples, bins=min(200, int(len(self.x))), histtype='bar', density=True, alpha=0.5, range=(pruned_durations.min(), pruned_durations.max()))
        #predictions = np.exp(self.gmm.score_samples(np.ravel(f_axis).astype(np.float64).reshape(-1,1)))
        #plt.plot(f_axis, predictions)
        if show_all:
            _, bins, patches = plt.hist(self.x, bins=min(200, int(len(self.x))), histtype='bar', density=True, alpha=0.5, range=(pruned_durations.min(), pruned_durations.max()))
            max_height = max([i.get_height() for i in patches])
            plt.ylim(0, max_height*1.1)
            for i in range(len(self.gmm.weights_)):
                plt.plot(f_axis,self.gmm.weights_[i]*stats.norm.pdf(f_axis,self.gmm.means_[i],np.sqrt(self.gmm.covariances_[i])).ravel(),
                        linestyle='dashed')

            plt.scatter(self.gmm.means_, [0]*len(self.gmm.means_), marker='o', color='red') 
            
            plt.rcParams['agg.path.chunksize'] = 10000
            plt.grid()

        if other_gmm:
            plt.plot(f_axis, other_gmm.get_pdf_values(f_axis))
        plt.show()

class AutoGMM(GMM):
    def __init__(self, start_end_event_log, percentile=False,
                max_gaussians=100, min_data_per_gaussian=5,
                min_relative_improvement=0.1,
                min_relative_improvement_period=3,
                max_fitting=5):
        self.n = 1
        super().__init__(start_end_event_log, self.n, percentile)
        goodness = self.goodness()
        max_gaussians = int(min(max_gaussians, len(self.x)/min_data_per_gaussian))
        non_improvement_period = 0
        for i in range(2, max_gaussians+1):
            self.get_gmm(self.x, i)
            super().__init__(start_end_event_log, i, percentile)
            new_goodness = self.goodness()
            self.n = i
            #print(self.n, goodness, (goodness - new_goodness) / goodness)
            if (goodness - new_goodness) / goodness < min_relative_improvement:
                non_improvement_period += 1
                if non_improvement_period >= min_relative_improvement_period:
                    break
            else:
                goodness = new_goodness
                non_improvement_period = 0
            if max_fitting > new_goodness:
                break        

In [ ]:
for concept in pandas.unique(start_end_event_log['concept:name']):
    concept_log = start_end_event_log[start_end_event_log['concept:name'] == concept]
    gmm = GMM(concept_log, 10, 95)
    gmm.plot_gmm(percentile=False)

    print(concept)

In [ ]:
activity_log = start_end_event_log[start_end_event_log['concept:name'] == 'W_Afhandelen leads']
activity_log = activity_log[activity_log['duration_seconds'] <= np.percentile(activity_log['duration_seconds'], 95)]

for i in range(1, activity_log.shape[0]+1):
    gmm2 = GMM(activity_log, i)
    print(i, gmm2.goodness())
    gmm2.plot_gmm()

In [ ]:
gmm2 = AutoGMM(activity_log, percentile=95)
print(gmm2.n, gmm2.goodness())
gmm2.plot_gmm()

In [ ]:
activity_log = start_end_event_log[start_end_event_log['concept:name'] == 'W_Afhandelen leads']
#activity_log = activity_log[activity_log['duration_seconds'] <= np.percentile(activity_log['duration_seconds'], 95)]
resource_log = activity_log[activity_log['org:resource'] == '11019']

for i in range(1, resource_log.shape[0]+1):
    gmm2 = GMM(resource_log, i)
    print(i, gmm2.goodness())
    gmm2.plot_gmm()

In [ ]:
artificial_log = pandas.DataFrame([1, 11, 21, 31, 41, 51, 61, 71, 81, 91], columns=['duration_seconds'])

for i in range(1, artificial_log.shape[0]+1):
    gmm2 = GMM(artificial_log, i)
    print(i, gmm2.goodness())
    gmm2.plot_gmm()

# Wasserstein distance

ot.wasserstein_1d is crap: does not really weight

In [ ]:
class GMMsWassersteinDistance:
    def __init__(self, gmm1, gmm2):
        self.gmm_1, self.gmm_2 = gmm1, gmm2

    def calculate(self, n=1000, std_devs=3):
        return self._get_wasserstein_distance(n, std_devs)

    def _get_sample_range_2(self, gmm, std_devs):
        model_mean = np.sum([mean*weight for weight, mean in zip(gmm.gmm.weights_, gmm.gmm.means_)])
        model_variance = np.sum([weight * variance for weight, variance in zip(gmm.gmm.weights_, gmm.gmm.covariances_)])
        model_std_dev = np.sqrt(model_variance)
        return model_mean - std_devs * model_std_dev, model_mean + std_devs * model_std_dev

    def _get_sample_range(self, gmm, std_devs):
        return np.min(gmm.x), np.max(gmm.x)

    def _sample_ppf(self, gmm, sample_range, n):
        x = np.linspace(sample_range[0], sample_range[1], n)
        #x = np.arange(sample_range[0], sample_range[1], 0.5)
        y = np.exp(gmm.gmm.score_samples(np.ravel(x).astype(np.float64).reshape(-1,1)))
        #Below: required for ot only
        #y = np.exp(gmm.gmm.score_samples(np.ravel(x).astype(np.float64).reshape(-1,1))) * (sample_range[1] - sample_range[0]) / n
        # either len(x) or n
        return x, y


    def _get_wasserstein_distance(self, n, std_devs):
        #sample_ranges = [self._get_sample_range(self.gmm_1, std_devs),
        #                 self._get_sample_range(self.gmm_2, std_devs),
        #                 #self._get_sample_range_2(self.gmm_1, std_devs),
        #                 #self._get_sample_range_2(self.gmm_2, std_devs)
        #                ]
        #sample_range = min(zip(*sample_ranges))[0], max(zip(*sample_ranges))[1]
        #print(sample_range)
        #u_values, u_weights = self._sample_ppf(self.gmm_1, sample_range, n)
        #v_values, v_weights = self._sample_ppf(self.gmm_2, sample_range, n)
        #wd_1 = stats.wasserstein_distance(u_values, v_values, u_weights, v_weights)
        
        #sample only from ppfs
        gmm_1_samples, _ = self.gmm_1.gmm.sample(n)
        gmm_2_samples, _ = self.gmm_2.gmm.sample(n)
        wd_2 = stats.wasserstein_distance(gmm_1_samples.ravel(), gmm_2_samples.ravel())
        #return wd_1, wd_2
        return wd_2

In [ ]:
gmm1 = GMM(start_end_event_log[start_end_event_log['concept:name'] == 'W_Afhandelen leads'], 5, 95)
gmm2 = GMM(start_end_event_log[start_end_event_log['concept:name'] == 'W_Completeren aanvraag'], 50, 95)


In [ ]:
wd = GMMsWassersteinDistance(gmm1, gmm2)
wd.calculate()

# Split

- We use activity: W_Afhandelen leads
- Afterwards we split by all resources with > 15 samples + all remaining resources
- Compare: ?


In [ ]:
activity_log = start_end_event_log[start_end_event_log['concept:name'] == 'W_Afhandelen leads']
activity_log = activity_log[activity_log['duration_seconds'] <= np.percentile(activity_log['duration_seconds'], 95)]

gmm1 = AutoGMM(activity_log)
resources = list(activity_log['org:resource'].value_counts().reset_index()['org:resource'])
gmm1.plot_gmm()

min_wd, max_wd = (np.inf, np.inf), (0, 0)
for resource in resources:
    activity_resource_log = activity_log[activity_log['org:resource'] == resource]
    if activity_resource_log.shape[0] >= 15:
        gmm2 = AutoGMM(activity_resource_log)
        wd = GMMsWassersteinDistance(gmm1, gmm2).calculate(10000)
        print(resource, activity_resource_log.shape[0], wd, gmm2.goodness())
        #gmm2.plot_gmm(gmm1)
        if wd > max_wd[0]:
            max_wd = wd, gmm2, activity_resource_log.shape[0], resource
        if wd < min_wd[0]:
            min_wd = wd, gmm2, activity_resource_log.shape[0], resource

print('min wd', min_wd)
min_wd[1].plot_gmm(gmm1)
print('max wd', max_wd)
max_wd[1].plot_gmm(gmm1)

In [ ]:
activity_log = start_end_event_log[start_end_event_log['concept:name'] == 'W_Afhandelen leads']
activity_log = activity_log[activity_log['duration_seconds'] <= np.percentile(activity_log['duration_seconds'], 95)]

amounts = activity_log['case:AMOUNT_REQ_start'].astype(int)
print(amounts.median())
amount_low_log = activity_log[activity_log['case:AMOUNT_REQ_start'].astype(int) <= amounts.median()]
amount_high_log = activity_log[activity_log['case:AMOUNT_REQ_start'].astype(int) > amounts.median()]


gmm1 = AutoGMM(activity_log)
gmm_high = AutoGMM(amount_high_log)
high_wd = GMMsWassersteinDistance(gmm1, gmm_high).calculate()

gmm_low = AutoGMM(amount_low_log)
low_wd = GMMsWassersteinDistance(gmm1, gmm_low).calculate()

gmm1.plot_gmm()

print('low amount wd', low_wd)
gmm_low.plot_gmm(gmm1)

print('high amount wd', high_wd)
gmm_high.plot_gmm(gmm1)

In [ ]:
class SplittingCriterion:
    def __init__(self, x_name, data, min_class_size):
        self.x_name = x_name
        self.data = data
        self.min_class_size = min_class_size

    def _get_optimal_split_continous(self):
        self._get_optimal_split_categorical()

    def _get_optimal_split_categorical(self):
        unique_values = self.data[self.x_name].unique()
        max_wd = 0
        best_split = None
        for unique_value in tqdm(unique_values):
            left = self.data[self.data[self.x_name] == unique_value]
            right = self.data[self.data[self.x_name] != unique_value]
            if left.shape[0] < self.min_class_size or right.shape[0] < self.min_class_size:
                continue
            left_gmm = AutoGMM(left)
            right_gmm = AutoGMM(right)
            wd = GMMsWassersteinDistance(left_gmm, right_gmm).calculate()
            if wd > max_wd:
                max_wd = wd
                best_split = wd, unique_value, left, right, left_gmm, right_gmm
        return best_split
            
    def get_optimal_split(self):
        if pandas.api.types.is_integer_dtype(self.data[self.x_name]) \
                or pandas.api.types.is_float_dtype(self.data[self.x_name]):
            return self._get_optimal_split_continous()
        else:
            return self._get_optimal_split_categorical()

class DecisionTree:
    def __init__(self, min_class_size=10, min_wd_gain=100):
        self.min_class_size = min_class_size
        self.min_wd_gain = min_wd_gain

    def _rec_fit(self, event_log, gmm):
        best_split = (0, 0, None, None, None, None)
        best_col = None
        for col in tqdm(event_log.columns):
            if col == 'duration_seconds':
                continue
            #print(col)
            split = SplittingCriterion(col, event_log, self.min_class_size).get_optimal_split()
            if split != None:
                if best_split[0] < split[0]:
                    best_split = split
                    best_col = col
        if best_split[0]:
            print(best_col, best_split[0])
            left_result, right_result = None, None
            if GMMsWassersteinDistance(gmm, best_split[4]).calculate() > self.min_wd_gain:
                left_result = self._rec_fit(best_split[2], best_split[4])
            if GMMsWassersteinDistance(gmm, best_split[5]).calculate() > self.min_wd_gain:
                right_result = self._rec_fit(best_split[3], best_split[5])
            return [(best_col, best_split[1], best_split[0]), left_result, right_result]
        else:
            return None
                    
    
    def fit(self, start_end_event_log):
        gmm = AutoGMM(start_end_event_log)
        return self._rec_fit(start_end_event_log, gmm)

                
        

In [ ]:
relevant_log = start_end_event_log[['org:resource', 'concept:name', 'case:AMOUNT_REQ_start', 'duration_seconds']]
dt = DecisionTree()
dt.fit(relevant_log)
